In [1]:
import os
os.environ['HF_HOME'] = '/data1/malto/cache'

In [2]:
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from pathlib import Path

BASE_DIR = Path("/data1/malto/shroom/")
BATCH_SIZE = 48
NUM_EPOCHS_SYN = 2
NUM_EPOCHS_GPT = 1
NUM_EPOCHS_TRUE = 5

BASE_DIR = Path("/data1/malto/shroom/")

checkpoint = "microsoft/deberta-large-mnli"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [3]:
def preprocess_function(examples): # not batched
    model_inputs = tokenizer(examples['hyp'], examples['tgt'] if examples['ref'] != 'src' else examples['src'], truncation=True, max_length=80)
    model_inputs["labels"] = [1 if t == "Hallucination" else 0 for t in examples['labels']]
    return model_inputs

def get_label(examples): # not batched
    return {"label" : 1 if examples['p(Hallucination)'] > 0.5 else 0}

def compute_metrics(eval_pred):
    #print(eval_pred)
    accuracy = evaluate.load("accuracy")
    predictions, labels = eval_pred
    #print(predictions, labels)
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "Not Hallucination", 1: "Hallucination"}
label2id = {"Not Hallucination": 0, "Hallucination": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

/data1/malto/fborra/venv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification m

In [4]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

syntetic_test_size_split = 0.000000001

ds_mt = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_MT_SOLAR.model-agnostic.json")])
ds_dm = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_DM_SOLAR.model-agnostic.json")])
ds_pg = load_dataset("json", data_files=[str(BASE_DIR / f"train_labeled_PG_SOLAR.model-agnostic.json")])
ds_val = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-agnostic.json")])
ds_val_aware = load_dataset("json", data_files=[str(BASE_DIR / f"val.model-aware.json")])
ds_gpt = load_dataset("json", data_files=str(BASE_DIR / f"transformed_val_model_gpt.json"))

ds_mt = ds_mt.remove_columns([el for el in ds_mt['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_dm = ds_dm.remove_columns([el for el in ds_dm['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_pg = ds_pg.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)
ds_gpt = ds_gpt.remove_columns([el for el in ds_pg['train'].column_names if el not in ds_val['train'].column_names])['train'].train_test_split(test_size=syntetic_test_size_split)

ds_syn = concatenate_datasets([ds_mt['train'], ds_dm['train'], ds_pg['train']])

In [5]:
def train_with_dataset(ds, num_epochs):
    ds = ds.map(get_label).shuffle()
    training_args = TrainingArguments(
        output_dir=BASE_DIR / "checkpoint" / "sequential",
        learning_rate=1e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        report_to="none",
        save_strategy="no",
        logging_steps=1,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds.map(preprocess_function),
        eval_dataset=ds_val_aware.map(get_label).map(preprocess_function),
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

In [6]:
train_with_dataset(ds_syn, NUM_EPOCHS_SYN)

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

Map:   0%|          | 0/29997 [00:00<?, ? examples/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Train Loss,Train Accuracy
1,0.465700,No log,0.838622,0.588822
2,0.500100,No log,0.821942,0.588822


In [14]:
train_with_dataset(ds_gpt['train'], NUM_EPOCHS_GPT)

Epoch,Training Loss,Validation Loss,Train Loss,Train Accuracy
1,0.290300,No log,1.840922,0.684631


In [15]:
train_with_dataset(ds_val['train'], NUM_EPOCHS_TRUE)

Epoch,Training Loss,Validation Loss,Train Loss,Train Accuracy
1,0.000100,No log,2.070816,0.752495
2,0.000000,No log,2.376184,0.740519
3,0.000000,No log,2.406332,0.744511
4,0.000000,No log,2.429646,0.746507
5,0.000000,No log,2.440250,0.748503
